In [1]:
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET
from get_edgar_data import ParseXML
import os
from tqdm import tqdm

### Turn filings into dataframe

In [ ]:
class ParseXML:
    @staticmethod
    def calc_transactionAmounts(xmlpath):
        """Calculate the total transaction amount in $ of a giving form 4 in XML"""
        xml = ET.parse(xmlpath).getroot()
        total = 0

        if xml is None:
            return total

        nonDerivativeTransactions = xml.findall(
            "./nonDerivativeTable/nonDerivativeTransaction"
        )

        for t in nonDerivativeTransactions:
            # D for disposed or A for acquired
            action = t.find(
                "./transactionAmounts/transactionAcquiredDisposedCode/value"
            ).text
            # number of shares disposed/acquired
            shares = t.find("./transactionAmounts/transactionShares/value").text
            # price
            priceRaw = t.find("./transactionAmounts/transactionPricePerShare/value")
            price = 0 if priceRaw is None else priceRaw.text
            # set prefix to -1 if derivatives were disposed. set prefix to 1 if derivates were acquired.
            prefix = -1 if action == "D" else 1
            # calculate transaction amount in $
            amount = prefix * float(shares) * float(price)
            total += amount

        return round(total, 2)

    @staticmethod
    def calc_transactionPricePerShare(xmlpath):
        """Calculate the avg transaction price per share in $ of a giving form 4 in XML"""
        xml = ET.parse(xmlpath).getroot()
        if xml is None:
            return np.nan

        nonDerivativeTransactions = xml.findall(
            "./nonDerivativeTable/nonDerivativeTransaction"
        )

        prices = []
        shares = []
        for t in nonDerivativeTransactions:
            action = t.find(
                "./transactionAmounts/transactionAcquiredDisposedCode/value"
            ).text
            shareRaw = t.find("./transactionAmounts/transactionShares/value").text
            priceRaw = t.find("./transactionAmounts/transactionPricePerShare/value")
            if not priceRaw is None:
                prices.append(float(priceRaw.text) * float(shareRaw))
                shares.append(float(shareRaw))

        if (len(prices) > 0) and (len(shares) >0) and (np.sum(shares)!=0):
            return np.sum(prices) / np.sum(shares)
        else:
            return np.nan

    @staticmethod
    def calc_transactionShares(xmlpath):
        """Calculate the avg transaction price per share in $ of a giving form 4 in XML"""
        xml = ET.parse(xmlpath).getroot()

        if xml is None:
            return np.nan

        nonDerivativeTransactions = xml.findall(
            "./nonDerivativeTable/nonDerivativeTransaction"
        )

        shares = []
        for t in nonDerivativeTransactions:
            action = t.find(
                "./transactionAmounts/transactionAcquiredDisposedCode/value"
            ).text
            shareRaw = t.find("./transactionAmounts/transactionShares/value")
            prefix = -1 if action == "D" else 1
            if not shareRaw is None:
                shares.append(prefix * float(shareRaw.text))

        if len(shares) > 0:
            return np.sum(shares)
        else:
            return np.nan

    @staticmethod
    def calc_absTransactionShares(xmlpath):
        """Calculate the avg transaction price per share in $ of a giving form 4 in XML"""
        xml = ET.parse(xmlpath).getroot()
        if xml is None:
            return np.nan

        nonDerivativeTransactions = xml.findall(
            "./nonDerivativeTable/nonDerivativeTransaction"
        )

        shares = []
        for t in nonDerivativeTransactions:
            shareRaw = t.find("./transactionAmounts/transactionShares/value")
            if not shareRaw is None:
                shares.append(float(shareRaw.text))
        if len(shares) > 0:
            return np.sum(shares)
        else:
            return np.nan

In [ ]:
def get_filed_date(filingDir):
    """
    return format: str, 20180109
    """
    import re
    with open(filingDir + "/full-submission.txt", "r") as f:
        content = f.read()
    date = re.search("FILED AS OF DATE:[\s]*[0-9]{8}", content).group()[-8:]
    return pd.Timestamp(date)

filingDir = "./sec-edgar-filings/AAP\\4\\0000921895-18-000108"
xmlpath = filingDir + "/filing-details.xml"
ParseXML.calc_transactionAmounts(xmlpath=xmlpath)

### Get EDGAR stats for AAPL

In [ ]:
edgarDir = "./sec-edgar-filings/"
ticker = "BIO"
filingsDir = edgarDir + "/" + ticker + "/4/"
dates = []
transactionPricePerShare = []
transactionShares = []
absTransactionShares = []
transactionAmounts = []
filingName = []
for filingDir in os.listdir(filingsDir):
    dates.append(get_filed_date(filingsDir + "/" + filingDir))
    xmlpath = filingsDir + "/" + filingDir + "/filing-details.xml" 
    transactionPricePerShare.append(ParseXML.calc_transactionPricePerShare(xmlpath=xmlpath))
    transactionShares.append(ParseXML.calc_transactionShares(xmlpath=xmlpath))
    absTransactionShares.append(ParseXML.calc_absTransactionShares(xmlpath=xmlpath))
    transactionAmounts.append(ParseXML.calc_transactionAmounts(xmlpath=xmlpath))
    filingName.append(filingDir)

In [ ]:
df_stock = pd.read_csv(f"./stock_data/{ticker}.csv", index_col=0)
df_stock.index[0]

In [ ]:
df_data = pd.DataFrame.from_dict({
    "date": dates,
    "transactionPricePerShare": transactionPricePerShare, 
    "transactionShares": transactionShares, 
    "absTransactionShares": absTransactionShares, 
    "transactionAmounts": transactionAmounts,
    "filingName": filingName
})
df_data = df_data.set_index('date')

In [ ]:
# df_data.to_csv(f"./edgar_data/{ticker}.csv")
df_stats = df_data.groupby(level=0).sum()
df_stats['numTransactions'] = df_data.groupby("date")['transactionAmounts'].count()
# df_stats.to_csv(f"./stock_edgar_stats/{ticker}.csv")
df_stats.index = df_stats.index.strftime('%Y-%m-%d')
df_stock = pd.read_csv(f"./stock_data/{ticker}.csv", index_col=0)
# df_stock.merge(df_stats, left_index=True, right_index=True, how='outer').to_csv(f"./bt_stock_data/{ticker}.csv")

In [ ]:
df_stock.dropna().loc['2018-01-02']

In [ ]:
df_stats

In [ ]:
df_stock.merge(df_stats, left_index=True, right_index=True, how='outer')

In [ ]:
df_stock.describe()

In [ ]:
def gen_bt_stock_data(ticker, edgarDir="./sec-edgar-filings/", bt_stock_data_dir="./bt_stock_data/"):
    filingsDir = edgarDir + "/" + ticker + "/4/"
    dates = []
    transactionPricePerShare = []
    transactionShares = []
    absTransactionShares = []
    transactionAmounts = []
    filingName = []
    for filingDir in os.listdir(filingsDir):
        dates.append(get_filed_date(filingsDir + "/" + filingDir))
        xmlpath = filingsDir + "/" + filingDir + "/filing-details.xml" 
        if os.path.exists(xmlpath):
            transactionPricePerShare.append(ParseXML.calc_transactionPricePerShare(xmlpath=xmlpath))
            transactionShares.append(ParseXML.calc_transactionShares(xmlpath=xmlpath))
            absTransactionShares.append(ParseXML.calc_absTransactionShares(xmlpath=xmlpath))
            transactionAmounts.append(ParseXML.calc_transactionAmounts(xmlpath=xmlpath))
            filingName.append(filingDir)

    df_data = pd.DataFrame.from_dict({
        "date": dates,
        "transactionPricePerShare": transactionPricePerShare, 
        "transactionShares": transactionShares, 
        "absTransactionShares": absTransactionShares, 
        "transactionAmounts": transactionAmounts,
        "filingName": filingName
    })
    df_data = df_data.set_index('date')

    df_data.to_csv(f"./edgar_data/{ticker}.csv")
    df_stats = df_data.groupby(level=0).sum()
    df_stats['numTransactions'] = df_data.groupby("date")['transactionAmounts'].count()
    df_stats.to_csv(f"./stock_edgar_stats/{ticker}.csv")
    df_stats.index = df_stats.index.strftime('%Y-%m-%d')

    df_stock = pd.read_csv(f"./stock_data/{ticker}.csv", index_col=0)
    df_stock.dropna(inplace=True)
    df_stock.merge(df_stats, left_index=True, right_index=True, how='outer').to_csv(f"./bt_stock_data/{ticker}.csv")
    df_stock = df_stock.merge(df_stats, left_index=True, right_index=True, how='outer')
    df_stock.to_csv(f"{bt_stock_data_dir}/{ticker}.csv")



In [ ]:
for file in tqdm(os.listdir("./stock_data/")):
    ticker = file[:-4]
    try:
        gen_bt_stock_data(ticker)
    except Exception as e:
        print(ticker, e)
#almost 20min

In [ ]:
df0 = pd.read_csv(f"./bt_stock_data/AAL.csv", index_col=0)
bad_tickers = []
for file in tqdm(os.listdir("./bt_stock_data/")):
       df = pd.read_csv(f"./bt_stock_data/{file}", index_col=0)
       if len(df.index) != len(df0.index):
              bad_tickers.append(file[:-4])

In [ ]:
df0 = pd.read_csv(f"./bt_stock_data/AAL.csv", index_col=0)
date_df_dict = {}
for file in tqdm(os.listdir("./bt_stock_data/")):
    if file[:-4] in bad_tickers:
        continue
    df = pd.read_csv(f"./bt_stock_data/{file}", index_col=0)
    for date in df0.index:
        if date not in date_df_dict:
            date_df_dict[date] = df.loc[date]
        else:
            date_df_dict[date] = pd.concat([date_df_dict[date], df.loc[date]],axis=1)
for k,v in date_df_dict.items():
    date_df_dict[k] = v.T.reset_index().set_index('ticker')
    date_df_dict[k].drop(columns='index', inplace=True)
    v.to_csv(f"./bt_stock_data_crossection/{k}.csv")

In [16]:
date_df_dict = {}
cols = ["rank_transactionPricePerShare", "rank_transactionShares", "rank_absTransactionShares", "rank_transactionAmounts", "rank_numTransactions"]
prefix = "sum_decen_"
tcols = [prefix + col for col in cols]
for file in tqdm(os.listdir("./bt_stock_data_crossection1/")):
    date = file[:-4]
    df = pd.read_csv(f"./bt_stock_data_crossection1/{file}", index_col=0)
    for i in range(len(cols)):
        df[tcols[i]] = np.nan 
        df[tcols[i]][~np.isnan(df[cols[i]])] = np.abs(df[cols[i]] - 0.5).sum()/2
    date_df_dict[date] = df
    df.to_csv(f"./bt_stock_data_crossection2/{file}")

100%|██████████| 1008/1008 [00:11<00:00, 87.32it/s]


In [ ]:
# cols = ['transactionPricePerShare', 'transactionShares', 'absTransactionShares', 'transactionAmounts', 'numTransactions']
# tcols = ['rank_transactionPricePerShare', 'rank_transactionShares', 'rank_absTransactionShares', 'rank_transactionAmounts', 'rank_numTransactions']
# for k,v in date_df_dict.items():
#     tmp = v[cols].rank(pct=True)
#     tmp.columns = tcols
#     tmp = pd.concat([v,tmp], axis=1)
#     tmp.to_csv(f"./bt_stock_data_crossection1/{k}.csv")

#### Turn Crosssectional Fields into Time-Series

In [10]:
ticker_df_dict = {}
for k,v in tqdm(date_df_dict.items()):
    for ticker in v.index:
        if ticker not in ticker_df_dict:
            tmp = v.loc[[ticker]]
            tmp['date'] = k
            ticker_df_dict[ticker] = tmp
        else:
            tmp = v.loc[[ticker]]
            tmp['date'] = k
            ticker_df_dict[ticker] = pd.concat([ticker_df_dict[ticker], tmp])

In [11]:
for k,v in ticker_df_dict.items():
    tmp = v.reset_index().set_index('date')
    tmp.to_csv(f"./bt_stock_data1/{k}.csv")